In [ ]:
%matplotlib inline
from datetime import datetime
import flopy
import h5py
import matplotlib.pyplot as plt
import mf6adj
import numpy as np
import os
import pandas as pd
import pathlib as pl
import platform

In [ ]:
env_path = pl.Path(os.environ.get("CONDA_PREFIX", None))
assert env_path is not None, "Notebook must be run from the mf6adj Conda environment"

bin_path = "bin"
if "linux" in platform.platform().lower():
    lib_ext = ".so"
elif "darwin" in platform.platform().lower() or "macos" in platform.platform().lower():
    lib_ext = ".dylib"
else:
    bin_path = "Scripts"
    lib_ext = ".dll"
    exe_ext = ".exe"
lib_name = env_path / f"{bin_path}/libmf6{lib_ext}"

## Load the existing model

In [ ]:
# load the simple model setup

name = "sv"
existing_ws = pl.Path("../models/synthetic-valley-advanced-annual")
ws = existing_ws.parent / "synthetic-valley-adj"

In [ ]:
sim = flopy.mf6.MFSimulation.load(
    sim_name=name, sim_ws=existing_ws, write_headers=False
)

In [ ]:
nper = sim.tdis.nper.array

In [ ]:
gwf = sim.get_model(name)
nlay = gwf.dis.nlay.array

### Change the simulation workspace

In [ ]:
sim.set_sim_path(ws)

## Write the model files

In [ ]:
sim.write_simulation()

### Set up the adjoint

In [ ]:
pm_path = ws / "sfr_perfmeas.dat"
with open(pm_path, "w") as fpm:
    sfr_data = pd.DataFrame.from_records(gwf.sfr.packagedata.array)
    fpm.write("begin performance_measure swgw\n")
    for n in range(1, nper):
        for k, i, j in sfr_data.cellid.values:
            fpm.write(f"{n + 1} 1 {k + 1} {i + 1} {j + 1} sfr-1 direct 1.0 -1.0e+30\n")
    fpm.write("end performance_measure\n\n")

### Run the model

In [ ]:
bd = os.getcwd()
os.chdir(ws)

In [ ]:
forward_hdf5_name = "forward.hdf5"
start = datetime.now()

adj = mf6adj.Mf6Adj(pm_path.name, lib_name, logging_level="INFO")
adj.solve_gwf(hdf5_name=forward_hdf5_name)  # solve the standard forward solution
dfsum = adj.solve_adjoint()  # solve the adjoint state for each performance measure
adj.finalize()  # release components
duration = (datetime.now() - start).total_seconds()
print("took:", duration)

In [ ]:
os.chdir(bd)

### Plot the streamflow capture results

In [ ]:
result_hdf = ws / "adjoint_solution_swgw_forward.hdf5"
hdf = h5py.File(result_hdf, "r")
keys = list(hdf.keys())
keys.sort()
print(keys)

In [ ]:
grp = hdf["composite"]
arr = -1.0 * grp["wel6_q"][:] / (nper - 1)
arr[arr < 0.0] *= -1.0

In [ ]:
contour_color = "white"
contour_style = "-"

sv_gwt_contour_dict = {
    "linewidths": 0.75,
    "colors": contour_color,
    "linestyles": contour_style,
}

In [ ]:
six_panel_figsize = (17.15 / 2.541, 1.4 * 0.8333 * 17.15 / 2.541)
mosaic = [[0, 1, 2], [3, 4, "."]]

In [ ]:
with flopy.plot.styles.USGSMap():
    fix, axd = plt.subplot_mosaic(
        mosaic,
        figsize=six_panel_figsize,
        layout="constrained",
    )

    for k in range(nlay):
        ax = axd[k]
        ax.set_xlim(gwf.modelgrid.extent[:2])
        ax.set_ylim(gwf.modelgrid.extent[2:])
        ax.set_title(f"Layer {k + 1}")
        mm = flopy.plot.PlotMapView(
            model=gwf,
            ax=ax,
            extent=gwf.modelgrid.extent,
            layer=k,
        )
        mm.plot_array(arr, vmin=0, vmax=1, edgecolor="black", lw=0.25, cmap="jet")

        mm.plot_bc(package=gwf.sfr, color="cyan")
        mm.plot_bc(package=gwf.lak)

        cs = mm.contour_array(
            arr,
            masked_values=[0],
            levels=np.arange(0, 1, 0.1),
            **sv_gwt_contour_dict,
        )

        ax.clabel(
            cs,
            inline=True,
            fmt="%2.1f",
            fontsize=12,
            inline_spacing=0.75,
            colors="white",
        )